# MuMoT Demonstration Notebook: Michaelis Menten <a class="tocSkip">
    
*Aldo Estefano Encarnacion Segura, Department of Computer Science, University of Sheffield*


This is a demo for [MuMoT](https://github.com/DiODeProject/MuMoT), a software tool developed at the University of Sheffield as part of the [DiODe](http://diode.group.shef.ac.uk) project

In [ ]:
import mumot

mumot.setVerboseExceptions()

### Introduction to the Michaelis-Menten model

In this short demo we will analyse one of the simplest models of enzyme kinetics, the Michaelis-Menten model. We will explore the four-dimensional original model, and then we will focus on two modified equations from the model after reducing the dynamic description of the model.

The original paper was published in German in 1913 ([Michaelis & Menten, 1913](#references)). An English translation of the original article as well as a historical context and summary was published by Johnson & Goody ([2011](#references)). For a general overview of the model see Murray ([2002](#references), p.175).

The Michaelis-Menten dynamics describe product formation as a result of an enzymatic reaction. Substrate $S$ binds irreversibly with enzyme $E$ to form a complex $C$, which in turn delivers the product $P$ and the enzyme.


#### Michaelis-Menten original model

In [ ]:
model = mumot.parseModel(r"""
E + S -> C + \emptyset : k_f
C + \emptyset -> E + S : k_r
C + \emptyset -> E + P : k_c
""")

In [ ]:
model.show()

In [ ]:
model.showODEs()

We have declared the reactions of the model and MuMoT has derived the ordinary differential equations that correspond to the four components of our system. Each term is a different reaction that either increases or decreases a variable's concentration. 

The reaction scheme is usually portrayed as:  $S + E ⟷ C \rightarrow P + E$  

In [ ]:
model.visualise()

We can integrate our model to track how the concentration of each of our components change over time. The parameters used are arbitrarily chosen for demonstration purposes. We start with substrate $S$ and $E$ present and no $P$ nor the intermediary complex $C$, which are yet to be formed.

The integration shows that both the enzyme and the complex go through a transient phase in which their concentration dips or rises before returning to its initial state. On the other hand, the substrate has been irreversibly transformed into the product. 

In [ ]:
int0 = model.integrate(initWidgets = {'\k_c':[0.1, 0, 0.5, 0.1],
                                       '\k_f':[1, 0, 10, 0.1],
                                       '\k_r':[0.01, 0, 0.1, 0.01],
                                       'initialState':{'S': [0.5,0,1,0.1],'E': [0.5,0,1,0.1],'P': [0,0,1,0.1],'C': [0,0,1,0.1]}}, 
                        maxTime = 50)


By selecting just the state variables that we are interested in, we can observe with more clarity the evolution over time of the substrate and the product. 

In [ ]:
int1 = model.integrate(showStateVars=['S','P'],initWidgets = {'\k_c':[0.1, 0, 0.5, 0.1],
                                       '\k_f':[1, 0, 10, 0.1],
                                       '\k_r':[0.01, 0, 0.1, 0.01],
                                       'initialState':{'S': [0.5,0,1,0.1],'E': [0.5,0,1,0.1],'P': [0,0,1,0.1],'C': [0,0,1,0.1]}}, 
                        maxTime = 50)


We can also analyse the dynamics using bifurcation analysis as well as looking at the vector field of the system, which allows us to see how all solutions develop over time. However, our model is a four-dimensional system, which unfortunately, prevents us from performing bifurcation analysis and vector analysis, since these are features supported only on lower-dimensional systems in MuMoT at present.

We can however undertake more limited analyses of the full dynamics, and fuller analyses of reduced dynamics.

##### The effects of noise

We can analyse the effects of noise in our model by making use of the Stochastic Simulation Algorithm (SSA). The magnitude of noise depends on the system size, as usual.

In [ ]:
mssa=model.SSA(initWidgets = {'\k_c':[0.1, 0, 0.5, 0.1],
                                       '\k_f':[1, 0, 10, 0.1],
                                       '\k_r':[0.01, 0, 0.1, 0.01],
                                       'systemSize':[10,1,100,1],
                                       'initialState':{'S': [0.5,0,1,0.1],'E': [0.5,0,1,0.1],'P': [0,0,1,0.1],'C': [0,0,1,0.1]}}, 
                                       maxTime = 50, runs = 20,
                                       choose_yrange=[0,5.5])

#### Michaelis-Menten reduced dynamics 

In order to make use of some of MuMoT's feature we require a model with lower dimensions. Palsson ([2011](#references), p.81) shows how the original model can be analysed in terms of just $S$ and $C$ by using the relationship for the total amount of enzyme:

 $E_{o} = E + C$
 
By replacing $C$ in the equations of $S$ and $X$ we can obtain a two dimensional model that follows the dynamics of these two components in the same way our four-dimensional model did.

In [ ]:
model_reduced = mumot.parseModel(r"""
C + S -> S + S: k_f
C -> S: k_r
C -> \emptyset: k_c
S -> C: k_f*e_o
""")

In [ ]:
model_reduced.showODEs()

In [ ]:
int1 = model_reduced.integrate(initWidgets = {'\k_c':[0.1, 0, 0.5, 0.1],
                                       '\k_f':[1, 0, 10, 0.1],
                                       '\k_r':[0.01, 0, 0.1, 0.01],
                                       '\e_o':[0.5, 0, 1, 0.1],
                                       'initialState':{'S': [0.5,0,1,0.1],'C': [0.5,0,1,0.1]}}, 
                        maxTime = 50)


The `stream` function displays how, for our model, only a single solution exists at the origin, since both the substrate $S$ and the complex $C$ are depleted by the reaction.   

In [ ]:
vector1 = model_reduced.stream('S', 'C', showFixedPoints = True, initWidgets = {'\k_c':[0.1, 0, 0.5, 0.1],
                                       '\k_f':[1, 0, 10, 0.1],
                                       '\k_r':[0.01, 0, 10, 0.01],
                                       '\e_o':[0.5, 0, 1, 0.1],
                                       'initialState':{'S': [0.5,0,1,0.1],'C': [0,0,1,0.1]}})

##### The effects of noise

As before, we can analyse the effects of noise in our system.

In [ ]:
mssa_red=model_reduced.SSA(initWidgets = {'\k_c':[0.1, 0, 0.5, 0.1],
                                       '\k_f':[1, 0, 10, 0.1],
                                       '\k_r':[0.01, 0, 0.1, 0.01],
                                       '\e_o':[0.5, 0, 1, 0.1],
                                       'initialState':{'C': [1,0,1,0.1]},'S': [1,0,1,0.1]}, 
                        maxTime = 50, runs=20)

### References
<a id='references'></a>

* Michaelis L., Menten M. L. (1913) Die kinetik der invertinwirkung. _Biochemistry Z_ **49**, 333-369.
* Murray, J. D. (2002). _Mathematical Biology I: An Introduction_. 3rd edition. Springer-Verlag.
* Michaelis, L., Menten, M. L., Johnson, K. A., & Goody, R. S. (2011). The original Michaelis constant: translation of the 1913 Michaelis-Menten paper. _Biochemistry_ **50**(39), 8264–8269. https://doi.org/10.1021/bi201284u
* Palsson, B. (2011). Enzyme kinetics. In: _Systems Biology: Simulation of Dynamic Network States_ (pp. 80-81). Cambridge University Press. https://doi.org/10.1017/CBO9780511736179.007